In [ ]:
import ollama
import gradio as gr

MODEL="llama2"

In [2]:
messages = [
    {"role":"assistant","content":"You are an unhelpful assistant and will respond to user queries with sarcasm"},
    {"role":"user","content":"What is the capital of france"}
]

In [3]:
response = ollama.chat(model=MODEL, messages=messages)

In [ ]:
print(response['message']['content'])

## Building a chatbot

In [5]:
system_message = "You are an unhelpful assistant and will respond to user queries with sarcasm"

In [ ]:
# Simple chat code logic

print("Hello. I am your personal unhelpful assistant. How can I help you today?")
user_input = input("You: ")

while user_input != '/bye':
    messages.append({"role":"user","content":user_input})
    resp = ollama.chat(model=MODEL, messages=messages)
    messages.append({"role":"assistant","content":resp['message']['content']})
    print(resp['message']['content'])
    user_input = input("You: ")

print("You have exited the chat!")


In [30]:
# Chat function for Gradio

def chat_llama2(prompt, history):
    messages = [
        {"role":"assistant","content":system_message},
        {"role":"user","content":prompt}
    ]

    result = ollama.chat(
        model="llama2",
        messages=messages,
        stream=True         # allows us to stream the resulting response instead of displaying the entire output at once
        )
    
    # streaming the response from the model
    # this prints out each output token form the model to have the resulting text displayed in real time
    response = ""
    for chunk in result:
        if 'message' in chunk and 'content' in chunk['message']:
            chunk_text = chunk['message']['content']
            response += chunk_text or ""
            yield response

In [ ]:
gr.ChatInterface(
    fn=chat_llama2,
    type="messages",
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Type Question Here", container=False, scale=3),
    title="Unhelpful Assistant",
    description="Ask me a question and I will give you an unhelpful answer",
    theme="ocean",
    examples=["Are tomatoes a vegetable", "What is the capital of France", "Is yes no and a no a yes"],
    cache_examples=True
    ).launch()